In [141]:
import os
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [142]:
df = pd.read_parquet(os.path.join('data','jizdenky.parquet'))

In [143]:
df = df[df['odkud'].str.contains('Brno') | df['odkud'].str.contains('Praha')]
df = df[df['kam'].str.contains('Brno') | df['kam'].str.contains('Praha')]

In [144]:
df.sample(5)

,oscrapovano,prodejce,odkud,kam,odjezd,predstih,predstih_d,predstih_h,cena,prostredek,volnych_mist,obsazenost,jizdni_doba,vzdalenost,zpozdeni,cena_poznamka,den,prestupy,vlaky,mistenka_zdarma,nahradni_bus,volna_mista_economy,volna_mista_economy_plus,volna_mista_economy_business,volna_mista_premium
252108,2024-11-10 16:00:26,ČD,Praha hl.n.,Brno hl.n.,2025-01-09 10:08:00,59 days 18:07:34,59,1434.126111,305.0,vlak,NaN,[Předpokládané střední obsazení - doporučujeme...,212.0,257.0,[0],None,čt,0,[R 979 Vysočina],True,None,NaN,NaN,NaN,NaN
145932,2024-11-08 09:08:53,ČD,Brno hl.n.,Praha hl.n.,2024-11-14 18:39:00,6 days 09:30:07,6,153.501944,359.0,vlak,NaN,[Předpokládané střední obsazení - doporučujeme...,152.0,255.0,[0],None,čt,0,[rj 370 Vindobona],True,None,NaN,NaN,NaN,NaN
320011,2024-11-12 13:48:12,ČD,Praha hl.n.,Brno hl.n.,2024-11-18 14:06:00,6 days 00:17:48,6,144.296667,329.0,vlak,NaN,[Předpokládané střední obsazení - doporučujeme...,224.0,257.0,[0],None,po,0,[R 983 Vysočina],True,True,NaN,NaN,NaN,NaN
434864,2024-11-15 09:52:26,ČD,Praha hl.n.,Brno hl.n.,2024-11-30 18:03:00,15 days 08:10:34,15,368.176111,359.0,vlak,NaN,[Předpokládané střední obsazení - doporučujeme...,179.0,255.0,[0],None,so,0,[R 875 Svitava],True,None,NaN,NaN,NaN,NaN
441992,2024-11-15 13:59:08,ČD,Praha hl.n.,Brno hl.n.,2024-12-07 18:06:00,22 days 04:06:52,22,532.114444,359.0,vlak,NaN,[Předpokládané vysoké obsazení - doporučujeme ...,224.0,257.0,[0],None,so,0,[R 991 Vysočina],True,True,NaN,NaN,NaN,NaN


In [145]:
df = df[df['prestupy'] == 0]
df = df.explode('obsazenost')

In [146]:
df['jizdni_doba'].quantile([0.1,0.25,0.5,0.75,0.666,0.8,0.9])

0.100    148.0
0.250    152.0
0.500    155.0
0.750    186.0
0.666    179.0
0.800    188.0
0.900    224.0
Name: jizdni_doba, dtype: float64

In [147]:
nejdelsi = df['jizdni_doba'].quantile(0.666)
nejdelsi

179.0

In [148]:
df = df[df['jizdni_doba'] <= nejdelsi]

In [149]:
df = df[df['predstih_h'] <= 4]

In [150]:
df['spoj'] = df.apply(lambda row: f"""{row['prodejce']} {row['den']} {row['odjezd'].hour:02}:{row['odjezd'].minute:02}""", axis=1)

In [151]:
df['spoj'].sample(5)

153796    ČD pá 13:39
150412    ČD pá 12:03
403715    RJ čt 15:15
247688    RJ ne 15:15
45478     ČD ne 17:44
Name: spoj, dtype: object

In [152]:
df[df['predstih_h'] < 4].groupby('spoj')['volnych_mist'].min().nlargest(10)

spoj
RJ st 20:56    307.0
RJ út 20:56    296.0
RJ po 20:56    295.0
RJ pá 20:56    283.0
RJ so 20:56    238.0
RJ čt 20:56    227.0
RJ ne 06:15    220.0
RJ ne 07:09    219.0
RJ st 06:15    213.0
RJ út 18:15    208.0
Name: volnych_mist, dtype: float64

In [193]:
df[df['predstih_h'] < 4].groupby('spoj')['volnych_mist'].min().dropna().nsmallest(50)

spoj
RJ ne 11:15     0.0
RJ ne 13:15     0.0
RJ ne 14:09     0.0
RJ ne 15:15     0.0
RJ ne 16:09     0.0
RJ ne 16:15     0.0
RJ ne 17:15     0.0
RJ ne 18:09     0.0
RJ pá 10:09     0.0
RJ pá 12:09     0.0
RJ pá 14:09     0.0
RJ pá 15:15     0.0
RJ pá 16:09     0.0
RJ pá 16:15     0.0
RJ ne 19:09     1.0
RJ pá 08:09     1.0
RJ pá 09:15     1.0
RJ pá 13:15     1.0
RJ pá 17:15     2.0
RJ po 09:15     3.0
RJ so 10:09     3.0
RJ so 12:09     3.0
RJ čt 16:15     3.0
RJ čt 18:09     3.0
RJ ne 18:15     4.0
RJ čt 17:15     4.0
RJ ne 10:09     5.0
RJ čt 12:09     5.0
RJ so 09:15     6.0
RJ st 09:15     8.0
RJ so 08:09    10.0
RJ út 09:15    11.0
RJ čt 10:09    11.0
RJ so 11:15    12.0
RJ st 11:15    12.0
RJ čt 16:09    12.0
RJ pá 11:15    13.0
RJ st 12:09    13.0
RJ ne 12:09    14.0
RJ čt 07:15    14.0
RJ pá 07:15    15.0
RJ st 15:15    16.0
RJ so 14:09    17.0
RJ ne 09:15    18.0
RJ út 16:09    19.0
RJ čt 09:15    19.0
RJ st 16:15    22.0
RJ st 17:15    25.0
RJ čt 14:09    25.0
RJ so 07:15    

In [195]:
df[df['odkud'].str.contains('Brno')].groupby("spoj")['cena'].median().dropna().sort_values().head(20)

spoj
RJ út 19:09    179.0
RJ po 19:09    179.0
RJ ne 07:09    249.0
RJ út 18:09    249.0
RJ st 19:09    249.0
RJ st 18:09    249.0
RJ so 19:09    249.0
RJ so 18:09    249.0
RJ út 14:09    249.0
RJ po 18:09    249.0
RJ út 16:09    249.0
ČD út 21:39    269.0
ČD po 21:39    269.0
RJ čt 19:09    274.0
ČD st 03:10    295.0
ČD ne 03:10    295.0
RJ ne 08:09    299.0
RJ út 10:09    299.0
RJ út 08:09    299.0
RJ st 16:09    299.0
Name: cena, dtype: float64

In [197]:
df[df['odkud'].str.contains('Praha')].groupby("spoj")['cena'].median().dropna().sort_values().head(20)

spoj
RJ čt 20:56     99.0
RJ út 20:56     99.0
RJ st 20:56     99.0
RJ po 20:56     99.0
RJ ne 06:15    179.0
RJ st 06:15    179.0
RJ út 06:15    179.0
RJ po 18:15    179.0
RJ so 18:15    249.0
RJ so 20:56    249.0
RJ st 07:15    249.0
RJ so 06:15    249.0
RJ pá 20:56    249.0
RJ st 18:15    249.0
RJ út 07:15    249.0
RJ út 11:15    249.0
RJ pá 06:15    249.0
RJ po 06:15    249.0
RJ ne 07:15    249.0
RJ čt 07:15    249.0
Name: cena, dtype: float64

In [156]:
df[df['odkud'].str.contains('Brno')].groupby("spoj")['cena'].median().dropna().sort_values().tail(20)

spoj
ČD ne 16:39    449.0
ČD pá 12:39    449.0
ČD ne 18:39    449.0
ČD pá 10:39    449.0
ČD pá 08:39    449.0
ČD pá 14:39    449.0
RJ čt 12:09    469.0
RJ pá 08:09    469.0
RJ ne 12:09    469.0
RJ so 10:09    469.0
RJ pá 12:09    469.0
RJ ne 18:09    469.0
ČD út 00:29    479.0
ČD po 00:29    479.0
ČD st 00:29    479.0
ČD čt 00:29    519.0
ČD pá 00:29    519.0
ČD so 00:29    519.0
ČD ne 00:29    519.0
RJ pá 10:09    524.0
Name: cena, dtype: float64

In [157]:
df[df['odkud'].str.contains('Praha')].groupby("spoj")['cena'].median().dropna().sort_values().tail(20)

spoj
ČD pá 09:44    465.0
RJ pá 09:15    469.0
RJ pá 17:15    469.0
RJ so 09:15    469.0
ČD pá 15:44    475.0
ČD pá 16:44    475.0
ČD pá 12:44    475.0
ČD pá 13:44    475.0
ČD pá 14:03    475.0
ČD pá 14:44    475.0
ČD po 00:40    479.0
RJ pá 15:15    489.0
RJ ne 17:15    499.0
ČD út 00:40    519.0
RJ pá 13:15    519.0
ČD st 00:40    519.0
ČD čt 00:40    519.0
ČD ne 00:40    519.0
ČD so 00:40    519.0
ČD pá 00:40    545.0
Name: cena, dtype: float64

In [203]:
df[df['prodejce'] == 'ČD'].groupby('spoj')['obsazenost'].agg(pd.Series.mode)

spoj
ČD ne 00:03                         Předpokládané nízké obsazení
ČD ne 00:29                         Předpokládané nízké obsazení
ČD ne 00:40                         Předpokládané nízké obsazení
ČD ne 03:10                         Předpokládané nízké obsazení
ČD ne 05:39                         Předpokládané nízké obsazení
ČD ne 05:44                         Předpokládané nízké obsazení
ČD ne 06:39                         Předpokládané nízké obsazení
ČD ne 06:44                         Předpokládané nízké obsazení
ČD ne 07:39                         Předpokládané nízké obsazení
ČD ne 07:44                         Předpokládané nízké obsazení
ČD ne 08:03                         Předpokládané nízké obsazení
ČD ne 08:39    Předpokládané střední obsazení - doporučujeme ...
ČD ne 08:44                         Předpokládané nízké obsazení
ČD ne 09:39    [Předpokládané střední obsazení - doporučujeme...
ČD ne 09:44    Předpokládané vysoké obsazení - doporučujeme v...
ČD ne 10:03    Předp

In [205]:
df[df['prodejce'] == 'RJ'].groupby('spoj')['volnych_mist'].median()

spoj
RJ ne 06:15    220.0
RJ ne 07:09    219.0
RJ ne 07:15    183.0
RJ ne 08:09    110.0
RJ ne 09:15     18.5
RJ ne 10:09      5.5
RJ ne 11:15      1.0
RJ ne 12:09     14.0
RJ ne 13:15     10.0
RJ ne 14:09      0.0
RJ ne 15:15      0.0
RJ ne 16:09      0.0
RJ ne 16:15      0.0
RJ ne 17:15      0.5
RJ ne 18:09      1.0
RJ ne 18:15      4.0
RJ ne 19:09      2.5
RJ ne 20:56    149.0
RJ po 06:15     70.0
RJ po 07:09    171.0
RJ po 07:15     50.0
RJ po 08:09     76.0
RJ po 09:15      3.5
RJ po 10:09     80.0
RJ po 11:15     69.0
RJ po 12:09     61.5
RJ po 13:15    148.0
RJ po 14:09     54.0
RJ po 15:15    136.0
RJ po 18:09    124.0
RJ po 18:15    159.0
RJ po 19:09    210.5
RJ po 20:56    296.0
RJ pá 06:15     72.0
RJ pá 07:09     43.0
RJ pá 07:15     15.0
RJ pá 08:09      3.0
RJ pá 09:15      1.0
RJ pá 10:09      1.0
RJ pá 11:15     15.0
RJ pá 12:09      0.5
RJ pá 13:15      2.0
RJ pá 14:09      0.0
RJ pá 15:15      1.0
RJ pá 16:09      0.0
RJ pá 16:15      3.0
RJ pá 17:15      2.0
RJ pá 18

In [159]:
df.groupby('obsazenost').size()

obsazenost
Předpokládané nízké obsazení                                              543
Předpokládané střední obsazení - doporučujeme včasné pořízení místenky    456
Předpokládané vysoké obsazení - doporučujeme včasné pořízení místenky     219
dtype: int64

In [160]:
df[df['obsazenost'].isin(['Téměr vyprodaná místa k sezení','Předpokládané vysoké obsazení - doporučujeme včasné pořízení místenky'])].groupby('spoj').size() / df[df['obsazenost'].notnull()].groupby('spoj').size().sample(20)

spoj
ČD ne 00:03    NaN
ČD ne 08:44    NaN
ČD ne 09:39    NaN
ČD ne 09:44    NaN
ČD ne 10:44    NaN
ČD ne 11:39    NaN
ČD ne 12:39    NaN
ČD ne 12:44    NaN
ČD ne 13:39    NaN
ČD ne 13:44    NaN
ČD ne 14:03    NaN
ČD ne 14:39    NaN
ČD ne 15:39    NaN
ČD ne 16:03    NaN
ČD ne 16:39    NaN
ČD ne 17:39    NaN
ČD ne 18:44    NaN
ČD po 00:03    NaN
ČD po 00:29    NaN
ČD po 00:40    NaN
ČD po 09:39    NaN
ČD po 09:44    NaN
ČD po 10:44    NaN
ČD po 11:44    NaN
ČD po 14:39    NaN
ČD po 16:03    NaN
ČD pá 00:29    NaN
ČD pá 03:10    NaN
ČD pá 07:39    NaN
ČD pá 08:03    NaN
ČD pá 08:39    1.0
ČD pá 09:39    NaN
ČD pá 09:44    NaN
ČD pá 10:39    NaN
ČD pá 10:44    NaN
ČD pá 11:39    NaN
ČD pá 12:03    NaN
ČD pá 12:39    NaN
ČD pá 12:44    NaN
ČD pá 13:39    NaN
ČD pá 13:44    NaN
ČD pá 14:03    NaN
ČD pá 14:44    NaN
ČD pá 15:39    NaN
ČD pá 15:44    NaN
ČD pá 16:03    NaN
ČD pá 16:39    NaN
ČD pá 16:44    NaN
ČD pá 17:39    NaN
ČD pá 17:44    NaN
ČD pá 18:03    NaN
ČD so 00:29    NaN
ČD so 0

In [161]:
df[df['prostredek'] == 'autobus']

,oscrapovano,prodejce,odkud,kam,odjezd,predstih,predstih_d,predstih_h,cena,prostredek,volnych_mist,obsazenost,jizdni_doba,vzdalenost,zpozdeni,cena_poznamka,den,prestupy,vlaky,mistenka_zdarma,nahradni_bus,volna_mista_economy,volna_mista_economy_plus,volna_mista_economy_business,volna_mista_premium,spoj


In [162]:
df = df[df['prostredek'] != 'autobus']

In [163]:
def tyden(frejm):
    frame = frejm.copy()
    frame['hodina'] = frame['odjezd'].dt.hour
    frame['blok'] = frame['hodina'] // 4
    vysledek = frame.groupby(['den','blok'])['volnych_mist'].median().reset_index()
    bloky = {
        0: '00:00-03:59',
        1: '04:00-07:59',
        2: '08:00-11:59',
        3: '12:00-15:59',
        4: '16:00-19:59',
        5: '20:00-23:59'
    }
    poradi = {'po': 1, 'út': 2, 'st': 3, 'čt': 4, 'pá': 5, 'so': 6, 'ne': 7}
    vysledek['část_dne'] = vysledek['blok'].map(bloky)
    vysledek['poradi_dne'] = vysledek['den'].map(poradi).astype(int)
    vysledek = vysledek.sort_values(by=['poradi_dne','část_dne'], ascending=True)
    return vysledek[['den','část_dne','volnych_mist']]

In [164]:
tyden(df[(df['prodejce'] == 'RJ') & (df['predstih_h'] < 4) & (df['odkud'].str.contains('Brno'))])

C:\Users\micha\AppData\Local\Temp\ipykernel_5348\2748219019.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  vysledek = frame.groupby(['den','blok'])['volnych_mist'].median().reset_index()


,den,část_dne,volnych_mist
4,po,04:00-07:59,171.0
5,po,08:00-11:59,79.0
6,po,12:00-15:59,59.0
7,po,16:00-19:59,206.0
20,út,04:00-07:59,86.0
21,út,08:00-11:59,49.5
22,út,12:00-15:59,90.0
23,út,16:00-19:59,110.0
16,st,04:00-07:59,122.0
17,st,08:00-11:59,92.0
